In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import scipy.optimize as spo

In [3]:
# OPEN_TIME = datetime.strptime('00:00:01', '%H:%M:%S')
# CLOSE_TIME = datetime.strptime('23:59:59', '%H:%M:%S')

OPEN_TIME = '00:00:00'
CLOSE_TIME = '24:00:00'

In [6]:

def get_opt_price_f(f_path):
    opt = pd.read_csv(f_path, encoding = 'CP949');
    opt = opt.rename(columns = {'일자' : 'BASE_DT', '시간' : 'TIME', '선물가격' : 'FUT', '지수' : 'IDX'});
    
    opt['BASE_DT'] = pd.to_datetime(opt['BASE_DT'], format = '%Y-%m-%d')
    opt['TIME'] = pd.to_datetime(opt['TIME'], unit = 'd').apply(lambda x : (x + timedelta(seconds=0.5)).replace(microsecond = 0)).dt.time
        
    opt = pd.melt(opt, id_vars=['BASE_DT', 'TIME'], var_name= 'INST', value_name='PR');
    fut = opt.loc[opt['INST'] == 'FUT'][['BASE_DT', 'TIME', 'PR']];
    idx = opt.loc[opt['INST'] == 'IDX'][['BASE_DT', 'TIME', 'PR']];
    
    opt = opt.loc[(opt['INST'] != 'IDX') & (opt['INST'] != 'FUT') & (opt['INST'] < '900')];
    opt = opt.rename(columns = {'INST' : 'STRK'}); 
    opt = opt.astype({'STRK' : 'float'}); 
    
    opt = opt.reset_index(drop = True).sort_values(['BASE_DT', 'TIME', 'STRK']);
    fut = fut.reset_index(drop = True).sort_values(['BASE_DT', 'TIME']);
    idx = idx.reset_index(drop = True).sort_values(['BASE_DT', 'TIME']);
    
    return opt, fut, idx;

In [4]:
def get_opt_price(base_ym, type, dir):
    f_path = dir + '/' + str(base_ym)
    
    if type.lower() == 'o' :
        f_path = f_path + '_O'
    elif type.lower() == 'h' :
        f_path = f_path + '_H'
    elif type.lower() == 'l' :
        f_path = f_path + '_L'
        
    #call option 가격 가져오기
    f_call = f_path + '_call.csv';
    [copt, fut_c, idx_c] = get_opt_price_f(f_call);
   
    #call option 가격 가져오기
    f_put = f_path + '_put.csv';
    [popt, fut_p, idx_p] = get_opt_price_f(f_put);
        
    #선물 가격 재생성
    fut = pd.merge(fut_c, fut_p, how = 'outer', on = ['BASE_DT', 'TIME'], suffixes = ('_C', '_P'));
    fut['PR'] = fut['PR_C'].fillna(fut['PR_P']);
    fut.drop(columns = ['PR_C', 'PR_P'], inplace = True);
    
    #지수 가격 재생성
    idx = pd.merge(idx_c, idx_p, how = 'outer', on = ['BASE_DT', 'TIME'], suffixes = ('_C', '_P'));
    idx['PR'] = idx['PR_C'].fillna(idx['PR_P']);
    idx.drop(columns = ['PR_C', 'PR_P'], inplace = True);
    
    copt = copt.reset_index(drop = True).sort_values(['BASE_DT', 'TIME', 'STRK']);
    popt = popt.reset_index(drop = True).sort_values(['BASE_DT', 'TIME', 'STRK']);
    fut = fut.reset_index(drop = True).sort_values(['BASE_DT', 'TIME']);
    idx = idx.reset_index(drop = True).sort_values(['BASE_DT', 'TIME']);
    
    return copt, popt, fut, idx;

In [5]:
def get_opt_atm(base_ym, dir, eqvl_srch_rng = 5):
    
    #시가
    [copt_o, popt_o, fut_o, idx_o] = get_opt_price(base_ym = base_ym, type = 'o', dir = d_dir);
    
    #고가
    [copt_h, popt_h, fut_h, idx_h] = get_opt_price(base_ym = base_ym, type = 'h', dir = d_dir);
    
    #저가
    [copt_l, popt_l, fut_l, idx_l] = get_opt_price(base_ym = base_ym, type = 'l', dir = d_dir);
    
    #종가
    [copt_c, popt_c, fut_c, idx_c] = get_opt_price(base_ym = base_ym, type = 'c', dir = d_dir);
   
    #선물/지수/옵션 데이터 합치기
    fut = pd.merge(fut_c, fut_o, how = 'left', on =['BASE_DT', 'TIME'], suffixes=('', '_O'));
    fut = pd.merge(fut, fut_h, how = 'left', on =['BASE_DT', 'TIME'], suffixes=('', '_H'));
    fut = pd.merge(fut, fut_l, how = 'left', on =['BASE_DT', 'TIME'], suffixes=('', '_L'));
    
    idx = pd.merge(idx_c, idx_o, how = 'left', on =['BASE_DT', 'TIME'], suffixes=('', '_O'));
    idx = pd.merge(idx, idx_h, how = 'left', on =['BASE_DT', 'TIME'], suffixes=('', '_H'));
    idx = pd.merge(idx, idx_l, how = 'left', on =['BASE_DT', 'TIME'], suffixes=('', '_L'));        

    fut_tmp = fut.rename(columns = {'PR' : 'FUT', 
                                    'PR_O' : 'FUT_O', 
                                    'PR_H' : 'FUT_H', 
                                    'PR_L' : 'FUT_L', });

    idx_tmp = idx.rename(columns = {'PR' : 'IDX',
                                    'PR_O' : 'IDX_O', 
                                    'PR_H' : 'IDX_H', 
                                    'PR_L' : 'IDX_L', });

    fut_idx = pd.merge(fut_tmp, idx_tmp, how = 'left', on =['BASE_DT', 'TIME']);
    
    copt = pd.merge(copt_c, copt_o, how = 'left', on =['BASE_DT', 'TIME', 'STRK'], suffixes=('', '_O'));
    copt = pd.merge(copt, copt_h, how = 'left', on =['BASE_DT', 'TIME', 'STRK'], suffixes=('', '_H'));
    copt = pd.merge(copt, copt_l, how = 'left', on =['BASE_DT', 'TIME', 'STRK'], suffixes=('', '_L'));

    popt = pd.merge(popt_c, popt_o, how = 'left', on =['BASE_DT', 'TIME', 'STRK'], suffixes=('', '_O'));
    popt = pd.merge(popt, popt_h, how = 'left', on =['BASE_DT', 'TIME', 'STRK'], suffixes=('', '_H'));
    popt = pd.merge(popt, popt_l, how = 'left', on =['BASE_DT', 'TIME', 'STRK'], suffixes=('', '_L'));

    
    copt_tmp = copt.rename(columns = {'PR' : 'C_PR', 
                                      'PR_O' : 'C_PR_O', 
                                      'PR_H' : 'C_PR_H', 
                                      'PR_L' : 'C_PR_L', });

    popt_tmp = popt.rename(columns = {'PR' : 'P_PR', 
                                      'PR_O' : 'P_PR_O', 
                                      'PR_H' : 'P_PR_H', 
                                      'PR_L' : 'P_PR_L', });

    opt = pd.merge(copt_tmp, popt_tmp, how = 'inner', on = ['BASE_DT', 'TIME', 'STRK']);
    opt = pd.merge(fut_idx, opt, how = 'left', on = ['BASE_DT', 'TIME'])

    #옵션 우세 찾기
    opt.loc[opt['C_PR'] > opt['P_PR'], 'OPT_WIN'] = 'C';
    opt.loc[opt['C_PR'] < opt['P_PR'], 'OPT_WIN'] = 'P';

    opt = opt.sort_values(by = ['BASE_DT', 'STRK', 'TIME'], ascending=[True, True, True]);
    opt['OPT_WIN'] = opt.groupby(['BASE_DT', 'STRK'])['OPT_WIN'].ffill();
    opt['OPT_WIN_PRV'] = opt.groupby(['BASE_DT', 'STRK'])['OPT_WIN'].shift(1);

    opt_1st_idx = opt.groupby(['BASE_DT', 'STRK']).head(1).index;
    opt.loc[opt_1st_idx, 'OPT_WIN_PRV'] = np.where(opt.loc[opt_1st_idx, 'C_PR_O'] > opt.loc[opt_1st_idx, 'P_PR_O'], 'C', 
                                        np.where(opt.loc[opt_1st_idx, 'C_PR_O'] < opt.loc[opt_1st_idx, 'P_PR_O'], 'P', np.nan))

    #만기 및 만기까지 남은 날
    opt['MATURITY'] = opt['BASE_DT'].max();
    opt['D-Days'] = (pd.to_datetime(opt['MATURITY'], format = '%Y-%m-%d') - pd.to_datetime(opt['BASE_DT'], format = '%Y-%m-%d')).dt.days;
    opt['BizD-Days'] = opt['BASE_DT'].rank(ascending = False, method = 'dense') - 1;

    opt = opt[['BASE_DT', 'MATURITY', 'D-Days', 'BizD-Days', 'TIME', 
               'FUT', 'FUT_O', 'FUT_H', 'FUT_L', 'IDX', 'IDX_O', 'IDX_H', 'IDX_L', 
               'STRK', 'C_PR', 'C_PR_O', 'C_PR_H', 'C_PR_L', 'P_PR', 'P_PR_O', 'P_PR_H', 'P_PR_L', 'OPT_WIN_PRV', 'OPT_WIN']];
    
    opt['EQVL_SUM'] = opt['C_PR'] + opt['P_PR'];

    opt['DIST'] = abs(opt['FUT'] - opt['STRK']);

    #atmn 찾기
    atm_srch_rng = 5
    opt_srch = opt[((opt['FUT'] - atm_srch_rng <= opt['STRK']) & (opt['STRK'] <= opt['FUT'] + atm_srch_rng)) & \
                    ((opt['FUT'] - atm_srch_rng <= opt['STRK']) & (opt['STRK'] <= opt['FUT'] + atm_srch_rng))].copy();
    opt_srch = opt_srch.sort_values(['BASE_DT','TIME', 'EQVL_SUM', 'DIST', 'STRK'], ascending=[True, True, True, True, False])
    opt_srch['RNK'] = opt_srch.groupby(['BASE_DT','TIME']).cumcount() + 1;
                                
    opt_atm = opt_srch[opt_srch['RNK'] == 1].drop(columns = ['RNK', 'DIST']);
    
    return opt_atm, copt, popt, fut, idx;

In [6]:
d_dir = './data'
# base_ym = list(range(202001, 202013))           \
#             + list(range(202101, 202113))       \
#             + list(range(202201, 202213))       \
#             + list(range(202301, 202313))       \
#             + list(range(202401, 202403))         

base_ym = list(range(202404, 202407))

base_ym.sort()
base_ym_begin = base_ym[0]
base_ym_end = base_ym[-1]

opt_atm = pd.DataFrame()
copt = pd.DataFrame()
popt = pd.DataFrame()
fut = pd.DataFrame()
idx = pd.DataFrame()

for base_ym_i in base_ym:
    print(base_ym_i)
    [opt_atm_i, copt_i, popt_i, fut_i, idx_i] = get_opt_atm(base_ym_i, d_dir)
    opt_atm = pd.concat([opt_atm, opt_atm_i])
    copt = pd.concat([copt, copt_i])
    popt = pd.concat([popt, popt_i])
    fut = pd.concat([fut, fut_i])
    idx = pd.concat([idx, idx_i])
    
opt_atm.to_csv("ATM_%d_%d.csv" % (base_ym[0], base_ym[-1]), index=False)


202404
202405
202406


In [7]:
def perf_ocrss(strtg_begin_buf_tm, strg_end_buf_tm, react_hold_dt, lc_dp, opt_atm):

    strg_end_buf_tm = 10*60 + strg_end_buf_tm    #전략 종료 시간 버퍼

    ent_dp = 0.5         # 진입 가격 변동
    clr_dp = 0.5         # 대응 가격 변동

    #atm 자료 시간 오름 차순으로 정렬
    opt_atm = opt_atm.sort_values(by = ['BASE_DT', 'TIME'], ascending=[True, True]);

    #A. 장 시작/종료 시간 추출
    mkt_tm_o = opt_atm.groupby(['BASE_DT'])[['BASE_DT', 'TIME']].head(1);
    mkt_tm_c = opt_atm.groupby(['BASE_DT'])[['BASE_DT', 'TIME']].tail(1);
    mkt_tm = pd.merge(mkt_tm_o, mkt_tm_c, how = 'inner', on = ['BASE_DT'], suffixes=['_O', '_C']);

    strtgy_opt = pd.merge(opt_atm, mkt_tm, how = 'inner', on = ['BASE_DT']);

    #Z. 전략 실행
    # atm_strk_cur
    # win_opt
    #손익
    tot_pl = 0
    fee = 0
    fee_rt = 0.003/100
    exec = pd.DataFrame(columns=['BASE_DT', 'TIME', 'POS_TYPE', 'TR', 'PL', 'FEE'])
    
    #position
    pos = 0
    pos_type = ''

    pr_ent = None # 진입가격
    react_hold_tm = datetime.min
    pr_h = -np.Inf
    pr_l = np.Inf

    for idx, atm_i in strtgy_opt.iterrows():
        tm_i = datetime.combine(atm_i['BASE_DT'], atm_i['TIME'])
        pr_i = atm_i['FUT']
        pr_o_i = atm_i['FUT_O']
        pr_h_i = atm_i['FUT_H']
        pr_l_i = atm_i['FUT_L']
        
        #장 시작 후, 일정 시간은 거래 skip
        mkt_o_tm = datetime.combine(atm_i['BASE_DT'], atm_i['TIME_O']) 
        if tm_i < mkt_o_tm + timedelta(seconds=strtg_begin_buf_tm):
            continue
        
        #장 종료 전, 청산
        mkt_c_tm = datetime.combine(atm_i['BASE_DT'], atm_i['TIME_C']) 
        if pos != 0 and mkt_c_tm - timedelta(seconds=strg_end_buf_tm) <= tm_i:
            
            pl_clr_i = np.sign(pos) * (pr_i - pr_ent)          
            fee_i = pr_i * fee_rt
            tot_pl = tot_pl + pl_clr_i - fee_i    
            
            exec_i = pd.DataFrame([{'BASE_DT' : tm_i.date(), 'TIME' : tm_i.time(), 'POS_TYPE' : pos_type, 'TR' : 'CLR', 'PL' : pl_clr_i, 'FEE' : fee_i}])
            exec = pd.concat([exec, exec_i])
            
            pos = 0              
            react_hold_tm = tm_i + timedelta(seconds = react_hold_dt)            
            pr_ent = None # 진입가격
            pr_h = -np.Inf
            pr_l = np.Inf

            continue
        
        #장 종료 전, 일정 시간은 거래 skip 
        if mkt_c_tm - timedelta(seconds=strg_end_buf_tm) <= tm_i:
            continue
                
        pr_h = max(pr_h, pr_h_i)
        pr_l = min(pr_l, pr_l_i)
        
        opt_win_prv_i = atm_i['OPT_WIN_PRV']
        opt_win_i = atm_i['OPT_WIN']
        
        #1. 손절 가격대이면, 손절(대응시간 변경, 가격 초기화, 손익 반영)
        pr_lc_i = 0
        if (pos > 0 and pr_l <= pr_ent - lc_dp):
            pr_lc_i = min(pr_ent - lc_dp, pr_o_i)
        elif (pos < 0 and pr_ent + lc_dp <= pr_h):
            pr_lc_i = max(pr_ent + lc_dp, pr_o_i)
            
        if pr_lc_i != 0: 
            
            pl_lc_i = -1 * abs(pr_lc_i - pr_ent)          
            fee_i = pl_lc_i * fee_rt
            tot_pl = tot_pl + pl_lc_i - fee_i
            
            exec_i = pd.DataFrame([{'BASE_DT' : tm_i.date(), 'TIME' : tm_i.time(), 'POS_TYPE' : pos_type, 'TR' : 'LC', 'PL' : pl_lc_i, 'FEE' : fee_i}])
            exec = pd.concat([exec, exec_i])
            
            pos = 0              
            react_hold_tm = tm_i + timedelta(seconds = react_hold_dt)
            pr_ent = pr_lc_i
            pr_h = pr_lc_i
            pr_l = pr_lc_i
            
            continue    
        
        #2. 대응 유보 시간 이내 경우, skip 
        if tm_i <= react_hold_tm:
            #2.1 put/call cross 변경 되었으면 대응방향 변경
            if (pos_type == 'L' and (opt_win_prv_i == 'C' and opt_win_i == 'P')):
                pos_type = 'S'
            elif (pos_type == 'S' and (opt_win_prv_i == 'P' and opt_win_i == 'C')):
                pos_type = 'L'     
                
            continue
        
        #3. 대응 유보 시간 이후인 경우,      
        pl_i = 0   
        
        # CALL에서 PUT 우세로 전환되었고, 
        if opt_win_prv_i == 'C' and opt_win_i == 'P':
            pos_type = 'S'

            #현재 SHORT 포지션이면, 
            if pos < 0:
                #아무 작업하지 않음
                continue
            
            #현재 LONG 포지션이면, 
            if pos > 0:
                #매도를 통한 손익 실현
                pl_i = (pr_i - pr_ent)
                fee_i = pr_i * fee_rt
                tot_pl = tot_pl + pl_i - fee_i

                exec_i = pd.DataFrame([{'BASE_DT' : tm_i.date(), 'TIME' : tm_i.time(), 'POS_TYPE' : pos_type, 'TR' : 'CLR', 'PL' : pl_i, 'FEE' : fee_i}])
                exec = pd.concat([exec, exec_i])
                
            #SHORT 포지션 진입
            pos = -1

            fee_i = pr_i * fee_rt
            tot_pl = tot_pl - fee_i
            exec_i = pd.DataFrame([{'BASE_DT' : tm_i.date(), 'TIME' : tm_i.time(), 'POS_TYPE' : pos_type, 'TR' : 'EL', 'PL' : 0, 'FEE' : fee_i}])            
            exec = pd.concat([exec, exec_i])
            
            react_hold_tm = tm_i + timedelta(seconds = react_hold_dt)
            pr_ent = pr_i
            pr_h = pr_i
            pr_l = pr_i                     
                    
            continue
        
        # PUT에서 CALL 우세로 전환되었고, 
        if opt_win_prv_i == 'P' and opt_win_i == 'C':
            pos_type = 'L'
        
            #현재 LONG 포지션이면, 
            if pos > 0:
                #아무 작업하지 않음
                continue
            
            #현재 SHORT 포지션이면, 
            if pos < 0:
                #매수를 통한 손익 실현
                pl_i = (pr_ent - pr_i)                
                fee_i = pr_i * fee_rt
                tot_pl = tot_pl + pl_i - fee_i

                exec_i = pd.DataFrame([{'BASE_DT' : tm_i.date(), 'TIME' : tm_i.time(), 'POS_TYPE' : pos_type, 'TR' : 'CLR', 'PL' : pl_i, 'FEE' : fee_i}])
                exec = pd.concat([exec, exec_i])
                            
            #LONG 포지션 진입
            pos = 1
            
            fee_i = pr_i * fee_rt
            tot_pl = tot_pl - fee_i
            exec_i = pd.DataFrame([{'BASE_DT' : tm_i.date(), 'TIME' : tm_i.time(), 'POS_TYPE' : pos_type, 'TR' : 'ES', 'PL' : 0, 'FEE' : fee_i}])         
            exec = pd.concat([exec, exec_i])
            
            react_hold_tm = tm_i + timedelta(seconds = react_hold_dt)
            pr_ent = pr_i
            pr_h = pr_i
            pr_l = pr_i 
                     
            continue       
    return [tot_pl, exec]

strtg_begin_buf_tm = 6         # 전략 시작 시간 버퍼
strg_end_buf_tm = 337.85   #전략 종료 시간 버퍼
react_hold_dt = 5*60 # 대응유보시간
ent_dp = 0.5         # 진입 가격 변동
clr_dp = 0.5         # 대응 가격 변동
lc_dp = 1        # 손절 가격 변동
[pl, exec] = perf_ocrss(strtg_begin_buf_tm, strg_end_buf_tm, react_hold_dt, lc_dp, opt_atm)
print(pl)
print(exec)
exec.to_csv("exec_%d_%d.csv" % (base_ym[0], base_ym[-1]), index=False)


/tmp/ipykernel_28508/3005593706.py:173: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  exec = pd.concat([exec, exec_i])


45.01541399999786
       BASE_DT      TIME POS_TYPE   TR    PL       FEE
0   2024-03-14  11:20:10        L   ES     0  0.011024
0   2024-03-14  11:38:40        S  CLR -0.05  0.011022
0   2024-03-14  11:38:40        S   EL     0  0.011022
0   2024-03-14  11:46:00        L  CLR -0.05  0.011024
0   2024-03-14  11:46:00        L   ES     0  0.011024
..         ...       ...      ...  ...   ...       ...
0   2024-06-13  13:47:50        L  CLR  -0.1  0.011326
0   2024-06-13  13:47:50        L   ES     0  0.011326
0   2024-06-13  13:59:10        S  CLR -0.05  0.011325
0   2024-06-13  13:59:10        S   EL     0  0.011325
0   2024-06-13  15:29:30        S  CLR  1.75  0.011272

[894 rows x 6 columns]


In [ ]:
def _perf_ocrss(x):

    strtg_begin_buf_tm = x[0]         # 전략 시작 시간 버퍼
    strg_end_buf_tm = x[1]   #전략 종료 시간 버퍼
    react_hold_dt = x[2] # 대응유보시간
    ent_dp = 0.5         # 진입 가격 변동
    clr_dp = 0.5         # 대응 가격 변동
    lc_dp = x[3]         # 손절 가격 변동
    [pl, exec] = perf_ocrss(strtg_begin_buf_tm, strg_end_buf_tm, react_hold_dt, lc_dp, opt_atm)
    
    return -1 * pl

def print_fun(x, convergence):
    x = np.round(x, 2)
    print("PL = %+.4f (strtg_begin_buf_tm = %+.2f, strg_end_buf_tm = %+.2f, react_hold_dt = %+.2f, lc_dp = %+.2f)" \
                      % (-_perf_ocrss(x), x[0], x[1], x[2], x[3]))
    
bnds = ((0, 60*60), (0, 60*60), (0, 60*60), (0, 4))

x0 = [3*60, 3*60, 1*60, 0.5]
strategy = ['best1bin', 'best1exp', 'rand1exp', 'randtobest1exp', 'currenttobest1exp', 'best2exp', 'rand2exp'
            , 'randtobest1bin', 'currenttobest1bin', 'best2bin', 'rand2bin', 'rand1bin']
x_init = x0

opt_best = np.inf
while True:
    
    for i in range(len(strategy)):
        strategy_i = strategy[i]
        print("------------------- strategy : %s -------------------" % (strategy_i))
        opt = spo.differential_evolution(func = _perf_ocrss, bounds = bnds, x0 = x_init, constraints=(), strategy = strategy_i
                                         , seed = i, disp = True, callback = print_fun, polish = False, updating = 'deferred', workers = -1, maxiter = 10000)
        x_init = opt.x

    if opt.fun >= opt_best:
        break
    opt_best = opt.fun
    
opt


In [8]:

strtg_begin_buf_tm = 9.88771391
strg_end_buf_tm = 339.08581299
react_hold_dt = 779.07759505
lc_dp = 1.00089428
[pl, exec] = perf_ocrss(strtg_begin_buf_tm, strg_end_buf_tm, react_hold_dt, lc_dp, opt_atm)
print(pl)
exec
exec.to_csv("exec_%d_%d.csv" % (base_ym[0], base_ym[-1]), index=False)



/tmp/ipykernel_28508/3005593706.py:173: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  exec = pd.concat([exec, exec_i])


58.3326593731344


In [ ]:
strtg_begin_buf_tm = 3*60         # 전략 시작 시간 버퍼
strg_end_buf_tm = 10*60 + 3*60    #전략 종료 시간 버퍼

react_hold_dt = 1*60 # 대응유보시간
ent_dp = 0.5         # 진입 가격 변동
clr_dp = 0.5         # 대응 가격 변동
lc_dp = 1         # 손절 가격 변동

#atm 자료 시간 오름 차순으로 정렬
opt_atm = opt_atm.sort_values(by = ['BASE_DT', 'TIME'], ascending=[True, True]);

#A. 장 시작/종료 시간 추출
mkt_tm_o = opt_atm.groupby(['BASE_DT'])[['BASE_DT', 'TIME']].head(1);
mkt_tm_c = opt_atm.groupby(['BASE_DT'])[['BASE_DT', 'TIME']].tail(1);
mkt_tm = pd.merge(mkt_tm_o, mkt_tm_c, how = 'inner', on = ['BASE_DT'], suffixes=['_O', '_C']);

strtgy_opt = pd.merge(opt_atm, mkt_tm, how = 'inner', on = ['BASE_DT']);


#Z. 전략 실행
# atm_strk_cur
# win_opt
#손익
pl = 0

#position
pos = 0
tr_type = ''

pr_ent = None # 진입가격
react_hold_tm = datetime.min
pr_h = -np.Inf
pr_l = np.Inf

for idx, atm in strtgy_opt.iterrows():
    tm = datetime.combine(atm['BASE_DT'], atm['TIME'])
    
    #장 시작 후, 일정 시간은 거래 skip
    mkt_o_tm = datetime.combine(atm['BASE_DT'], atm['TIME_O']) 
    if tm < mkt_o_tm + timedelta(seconds=strtg_begin_buf_tm):
        continue
    
    #장 종료 전, 일정 시간은 거래 skip 
    mkt_c_tm = datetime.combine(atm['BASE_DT'], atm['TIME_C']) 
    if mkt_c_tm - timedelta(seconds=strg_end_buf_tm) <= tm:
        continue
    
    pr = atm['FUT']
    pr_h = max(pr_h, atm['FUT_H'])
    pr_l = min(pr_l, atm['FUT_L'])
    
    opt_win_prv = atm['OPT_WIN_PRV']
    opt_win = atm['OPT_WIN']
    
    #1. 손절 가격대이면, 손절(대응시간 변경, 가격 초기화, 손익 반영)
    pr_lc = None
    if not (pr_ent is None):
        pr_lc = pr_ent - np.sign(pos) * lc_dp    
        
    if (pos > 0 and pr_l <= pr_lc) or (pos < 0 and pr_lc <= pr_h): 
        pos = 0
        pl = pl - lc_dp

        react_hold_tm = tm + timedelta(seconds = react_hold_dt)
        pr_ent = pr_lc
        pr_h = pr_lc
        pr_l = pr_lc
        
        print('[%s][%.2f] %s ' % (tm, pl, '손절') )

        continue    
    
    #2. 대응 유보 시간 이내 경우, skip 
    if tm <= react_hold_tm:
        #2.1 put/call cross 변경 되었으면 대응방향 변경
        if (tr_type == 'L' and (opt_win_prv == 'C' and opt_win == 'P')):
            tr_type = 'S'
        elif (tr_type == 'S' and (opt_win_prv == 'P' and opt_win == 'C')):
            tr_type = 'L'     
            
        # print('[%s][%.2f] %s ' % (tm, pl, 'PC Cross(유보)') )
       
        continue
    
    #3. 대응 유보 시간 이후인 경우,         
    
    # CALL에서 PUT 우세로 전환되었고, 
    if opt_win_prv == 'C' and opt_win == 'P':
        #현재 SHORT 포지션이면, 
        if pos < 0:
            #아무 작업하지 않음
            # print('[%s][%.2f] %s ' % (tm, pl, 'PC Cross(유지)') )
         
            continue
        
        #현재 LONG 포지션이면, 
        if pos > 0:
            #매도를 통한 손익 실현
            pl = pl + (pr - pr_ent)
            print('[%s][%.2f] %s ' % (tm, pl, '매도진입(w 청산)') )
        else : 
            print('[%s][%.2f] %s ' % (tm, pl, '매도진입') )

        #SHORT 포지션 진입
        pos = -1
        tr_type = 'S'
        
        react_hold_tm = tm + timedelta(seconds = react_hold_dt)
        pr_ent = pr
        pr_h = pr
        pr_l = pr 
                
                
        continue
    
    # PUT에서 CALL 우세로 전환되었고, 
    if opt_win_prv == 'P' and opt_win == 'C':
        #현재 LONG 포지션이면, 
        if pos > 0:
            #아무 작업하지 않음
            # print('[%s][%.2f] %s ' % (tm, pl, 'PC Cross(유지)') )
            continue
        
        #현재 SHORT 포지션이면, 
        if pos < 0:
            #매수를 통한 손익 실현
            pl = pl + -1 * (pr - pr_ent)
            print('[%s][%.2f] %s ' % (tm, pl, '매수진입(w 청산)') )
        else : 
            print('[%s][%.2f] %s ' % (tm, pl, '매수진입') )
        
        #LONG 포지션 진입
        pos = 1
        tr_type = 'L'
        
        react_hold_tm = tm + timedelta(seconds = react_hold_dt)
        pr_ent = pr
        pr_h = pr
        pr_l = pr 
                
        continue       


In [ ]:
def backtest_eqvl_i(ent_p, loss_cut, bdays_min, bdays_max, opt_eqvl):
    opt_eqvl_d_m = opt_eqvl[(pd.isna(opt_eqvl['BASE_DT_D1']) == False) 
                            & (opt_eqvl['TIME'] != OPEN_TIME) 
                            & (opt_eqvl['TIME'] != CLOSE_TIME)
                            & (bdays_min <= opt_eqvl['BizD-Days'])
                            & (opt_eqvl['BizD-Days'] < bdays_max) ].copy()

    opt_eqvl_d_m['GAP_FALL'] = (opt_eqvl_d_m['EQVL_PR_OPEN'] < opt_eqvl_d_m['EQVL_PR_CLOSE_D1'])

    #갭하락 장 시작 1분 종가 진입 가정
    x_opt_eqvl_gf = opt_eqvl_d_m[(opt_eqvl_d_m['GAP_FALL'] == True)]
    x_opt_eqvl_gf = x_opt_eqvl_gf.sort_values(['BASE_DT', 'TIME'], ascending = [True, True]) \
                                        .groupby(['BASE_DT'])                                       \
                                        .head(1).copy()

    #갭상승 시 OPEN가 보다 
    x_opt_eqvl_gr = opt_eqvl_d_m[(opt_eqvl_d_m['GAP_FALL'] == False)]
    x_opt_eqvl_gr = x_opt_eqvl_gr[(x_opt_eqvl_gr['EQVL_PR_OPEN'] + ent_p <= x_opt_eqvl_gr['EQVL_PR_O'])]   \
                                        .sort_values(['BASE_DT', 'TIME'], ascending = [True, True]) \
                                        .groupby(['BASE_DT'])                                       \
                                        .head(1).copy()

    x_opt_eqvl = pd.concat([x_opt_eqvl_gf, x_opt_eqvl_gr])
    
    if len(x_opt_eqvl) == 0:
        return pd.DataFrame() 
    
    x_opt_eqvl['BUY'] = x_opt_eqvl['EQVL_PR_O']
    x_opt_eqvl['LOSS_CUT'] = x_opt_eqvl['BUY'] + loss_cut 
    x_opt_eqvl['SELL'] = x_opt_eqvl.apply(lambda row : 
                                                row['EQVL_PR_C'] if row['EQVL_PR_H'] < row['LOSS_CUT'] else row['LOSS_CUT'] 
                                                , axis = 1)
    x_opt_eqvl['PL'] = x_opt_eqvl['BUY'] - x_opt_eqvl['SELL']
    x_opt_eqvl.sort_values(['BASE_DT', 'TIME'], ascending = [True, True], inplace = True)
    return x_opt_eqvl;


def backtest_eqvl(ent_p, loss_cut, bdays_rng, opt_eqvl):

    x_opt_eqvl = pd.DataFrame()
    for i in range(len(bdays_rng)):
        bdays_min_i = bdays_rng[i][0]
        bdays_max_i = bdays_rng[i][1]
        
        ent_p_i = ent_p[i]
        loss_cut_i = loss_cut[i]

        x_opt_eqvl_i = backtest_eqvl_i(ent_p_i, loss_cut_i, bdays_min_i, bdays_max_i, opt_eqvl)
        x_opt_eqvl = pd.concat([x_opt_eqvl, x_opt_eqvl_i])
    
    x_opt_eqvl.sort_values(['BASE_DT', 'TIME'], ascending = [True, True], inplace = True)
    
    return x_opt_eqvl;


In [ ]:
bdays_rng = ((25, 100),(15, 25), (10, 15), (5, 10), (0, 5))

def perf_eqvl(x):
    x = np.round(x, 2)

    pnlty = 0
    x_opt_eqvl = pd.DataFrame()
    n_bdays_rng = len(bdays_rng)

    ent_p = x[:n_bdays_rng]
    loss_cut = x[-n_bdays_rng:]

    MIN_ENT_P = 0.00
    pnlty = pnlty + np.sum(np.minimum(0, ent_p - MIN_ENT_P))
    ent_p[ent_p < MIN_ENT_P] = MIN_ENT_P

    MIN_LOSS_CUT = 0.01
    pnlty = pnlty + np.sum(np.minimum(0, loss_cut - MIN_LOSS_CUT))
    loss_cut[loss_cut <= MIN_LOSS_CUT] = MIN_LOSS_CUT
    
    x_opt_eqvl = backtest_eqvl(ent_p, loss_cut, bdays_rng, opt_eqvl)

    if len(x_opt_eqvl) == 0:
        return 0
    
    return -1 * (np.mean(x_opt_eqvl['PL']) + pnlty)

def print_fun(x, convergence):
    x = np.round(x, 2)
    print("PL = %+.4f (ep_25 = %+.2f, ep_15 = %+.2f, ep_10 = %+.2f, ep_05 = %+.2f, ep_00 = %+.2f, " 
                      "lc_25 = %+.2f, lc_15 = %+.2f, lc_10 = %+.2f, lc_05 = %+.2f, lc_00 = %+.2f)" \
                      % (-perf_eqvl(x), x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9]))

ent_p0 = 0.5;
loss_cut0 = 1;
ent_p25_0 = ent_p15_0 = ent_p10_0 = ent_p05_0 = ent_p00_0 = ent_p0;
loss_cut25_0 =loss_cut15_0 = loss_cut10_0 = loss_cut05_0 = loss_cut00_0 = loss_cut0;

x0 = [ent_p25_0, ent_p15_0, ent_p10_0, ent_p05_0, ent_p00_0, loss_cut25_0, loss_cut15_0, loss_cut10_0, loss_cut05_0, loss_cut00_0]
bnds = ((0, 10), (0, 10), (0, 10), (0, 10), (0, 10), 
        (0.05, 10), (0.05, 10), (0.05, 10), (0.05, 10), (0.05, 10))
# minimizer_kwargs = { "method" : "L-BFGS-B","bounds" : bnds }
# minimizer_kwargs = { "method" : "Nelder-Mead" }

# opt = spo.basinhopping(perf_eqvl, x0, callback = print_fun, minimizer_kwargs = minimizer_kwargs, seed = 1)
# opt
def constr_func(x):
    n_bdays_rng = len(bdays_rng)

    ent_p = x[:n_bdays_rng]
    loss_cut = x[-n_bdays_rng:]
    
    return list(np.subtract(ent_p[:(-1)], ent_p[1:])) + list(np.subtract(loss_cut[:(-1)], loss_cut[1:]))
    
nlc = spo.NonlinearConstraint(constr_func, 
                              [0, 0, 0, 0, 0, 0, 0, 0], 
                              [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf])


strategy = ['best1bin', 'best1exp', 'rand1exp', 'randtobest1exp', 'currenttobest1exp', 'best2exp', 'rand2exp'
            , 'randtobest1bin', 'currenttobest1bin', 'best2bin', 'rand2bin', 'rand1bin']
x_init = x0
x_init = [6.93, 2.86, 1.40, 0.99, 0.87, 9.35, 7.19, 0.47, 0.39, 0.39]

opt_best = np.inf
while True:
    
    for i in range(len(strategy)):
        strategy_i = strategy[i]
        print("------------------- strategy : %s -------------------" % (strategy_i))
        opt = spo.differential_evolution(func = perf_eqvl, bounds = bnds, x0 = x_init, constraints=nlc, strategy = strategy_i
                                         , seed = i, disp = True, callback = print_fun, polish = False, updating = 'deferred', workers = -1, maxiter = 10000)
        x_init = opt.x

    if opt.fun >= opt_best:
        break
    opt_best = opt.fun
    
opt

In [ ]:
perf_eqvl(x_init)

In [14]:
strtg_begin_buf_tm = 1121.45         # 전략 시작 시간 버퍼
strg_end_buf_tm = 471.38   #전략 종료 시간 버퍼
react_hold_dt = 311.50 # 대응유보시간
ent_dp = 0.5         # 진입 가격 변동
clr_dp = 0.5         # 대응 가격 변동
lc_dp = +1.46        # 손절 가격 변동
[pl, exec] = perf_ocrss(strtg_begin_buf_tm, strg_end_buf_tm, react_hold_dt, lc_dp, opt_atm)
print(pl)
print(exec)
exec.to_csv("exec_%d_%d.csv" % (base_ym[0], base_ym[-1]), index=False)


/tmp/ipykernel_15566/3005593706.py:173: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  exec = pd.concat([exec, exec_i])


52.99965679999856
       BASE_DT      TIME POS_TYPE   TR    PL       FEE
0   2024-03-14  11:20:10        L   ES     0  0.011024
0   2024-03-14  11:38:40        S  CLR -0.05  0.011022
0   2024-03-14  11:38:40        S   EL     0  0.011022
0   2024-03-14  11:46:00        L  CLR -0.05  0.011024
0   2024-03-14  11:46:00        L   ES     0  0.011024
..         ...       ...      ...  ...   ...       ...
0   2024-06-13  13:47:50        L  CLR  -0.1  0.011326
0   2024-06-13  13:47:50        L   ES     0  0.011326
0   2024-06-13  13:59:10        S  CLR -0.05  0.011325
0   2024-06-13  13:59:10        S   EL     0  0.011325
0   2024-06-13  15:27:10        S  CLR  1.75  0.011272

[794 rows x 6 columns]
